In [1]:
import geopandas as gpd
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import rasterio
import rasterio.features
import shapely
from shapely.geometry import box, Polygon
from tqdm import tqdm

In [2]:
tqdm.pandas()

# 1. Functions

In [3]:
def viz(cbg_sj, cbg_footprints, column, fp):

    # plot shapefile using the inferred area as the label for each CBG
    plt.clf()
    fig = plt.figure(figsize=(15, 15))
    ax = fig.gca()
    ax.set_aspect('equal')
    cbg_sj.plot(ax=ax, zorder=1, color='white', edgecolor="black")
    cbg_footprints.loc[cbg_footprints[column] == 1].plot(column='area', zorder=2, ax=ax, legend=True)
    plt.savefig(os.path.join(fp, 'outputs', 'plot_{}.png'.format(column)))

In [4]:
def get_footprint_gpd(mask, file_name, size):
    """
    Converts predictions from np.array to shapely polygons
    :param mask: (np.array) Footprint mask from which to generate the polygons
    :param file_name: (str) Name of the TIF file 
    :param size: (int) Resolution size [512, 1024]
    :return: (gpd.GeoDataFrame)
    """

    with rasterio.open(os.path.join(TIF_FP, f'{file_name}.tif')) as ds:
        t = ds.transform

    # Adjust for resolution
    if size == 512:
        factor = 1
    elif size == 1024:
        factor = 2
    elif size == 512 * 4:
        factor = 4 # CHECK
    else:
        raise NotImplemented('[ERROR] GPD Footprint --- Check resolution')

    shapes = rasterio.features.shapes(mask, connectivity=4)
    polygons = [shapely.geometry.Polygon(shape[0]["coordinates"][0]) for shape in shapes if shape[1] == 1]
    polygons = [shapely.affinity.affine_transform(geom, [t.a / factor, t.b, t.d, t.e / factor, t.xoff, t.yoff]) for geom
                in polygons]
    buildings = gpd.GeoDataFrame(geometry=polygons, crs='EPSG:26910')
    buildings = buildings.to_crs('EPSG:4326')
    return buildings

In [5]:
def collect_inferences(inferences_dir, size):
    inference_files = glob.glob(os.path.join(inferences_dir, '*.npy'))
    building_footprints = gpd.GeoDataFrame()
    for inference_file in tqdm(inference_files):
        file_name = inference_file.split(os.path.sep)[-1].replace('.npy', '')
        mask = np.load(inference_file)
        tile_buildings = get_footprint_gpd(mask, file_name, size)
        tile_buildings['tile'] = inference_file
        building_footprints = pd.concat([building_footprints, tile_buildings])

    return building_footprints

In [6]:
def assign_cbgs(building_footprints, cbg_sj):
    # Assign buildings to San Jose CBGs
    building_footprints = building_footprints.to_crs(cbg_sj.crs)
    building_footprints_cbg = gpd.sjoin(
        cbg_sj[['GEOID', 'geometry']], building_footprints, 
        how='right', predicate='contains')

    # Drop buildings outside of San Jose
    building_footprints_cbg = building_footprints_cbg.dropna(axis=0, subset=['GEOID'])
    return building_footprints_cbg

In [7]:
def process_buildings(building_footprints, zoning, sj_crs, verbose=True):
    # Merge buildings to connect buildings on tile edges
    if verbose:
        print('[INFO] Processing: Merging buildings on tile edges')
    geoms = building_footprints.geometry.unary_union
    building_footprints = gpd.GeoDataFrame(geometry=[geoms], crs='EPSG:4326')
    building_footprints = building_footprints.explode(index_parts=True).reset_index(drop=True)

    # Separate buildings
    if verbose:
        print('[INFO] Processing: Separating buildings')
    building_footprints = building_footprints.to_crs(crs=3857)
    if verbose:
        building_footprints.geometry = building_footprints.geometry.progress_apply(lambda x: x.buffer(-2))
    else:
        building_footprints.geometry = building_footprints.geometry.buffer(-2)
    building_footprints = building_footprints.explode(index_parts=False)
    if verbose:
        building_footprints.geometry = building_footprints.geometry.progress_apply(lambda x: x.buffer(2))
    else:
        building_footprints.geometry = building_footprints.geometry.buffer(2)
    building_footprints = building_footprints.to_crs(sj_crs)

    # Remove predictions on roads
    if verbose:
        print('[INFO] Processing: Removing predictions on roads')
    building_footprints = gpd.clip(building_footprints, zoning)

    return building_footprints

In [8]:
def compute_area(building_footprints, small_upper_threshold, large_upper_threshold, sj_crs):
    # Compute building area
    building_footprints = building_footprints.to_crs(crs=3857)
    building_footprints['area'] = building_footprints.area
    
    # Filter for buildings that are too small or too large
    building_footprints = building_footprints.loc[building_footprints['area'] > 15]
    if large_upper_threshold:
        building_footprints = building_footprints.loc[building_footprints['area'] < large_upper_threshold]

    # Define small/large buildings
    # add up sqft/m^2 for just small buildings, or for both large and small buildings
    building_footprints['small'] = building_footprints['area'] < small_upper_threshold
    building_footprints['large'] = 1 - building_footprints['small']
    
    building_footprints = building_footprints.to_crs(sj_crs)
    return building_footprints

In [9]:
def main(cbg_sj, zoning, inferences_dir, size, small_threshold, large_threshold, fp):

    # 1) Loop over each tile to convert the inference np.array to gpd
    # This step outputs a gpd of building footprints: building_footprints
    collected_footprints_file = os.path.join(fp, 'building_inferences')
    if not os.path.exists(collected_footprints_file):
        print('[INFO] Collecting building footprints')
        building_footprints = collect_inferences(inferences_dir, size)
        building_footprints.to_file(collected_footprints_file)
        print('[INFO] Number of buildings: {}'.format(len(building_footprints)))

    # 2) Vectorized processing of buildings at the cbg level using the gpd output from step 3
    #     - combine tile edge buildings
    #     - road filtering
    #     - building separation
    processed_footprints_file = os.path.join(fp, 'building_processed')
    if not os.path.exists(processed_footprints_file):
        print('[INFO] Loading building footprints')
        building_footprints = gpd.read_file(collected_footprints_file)
        print('[INFO] Number of buildings: {}'.format(len(building_footprints)))
        
        print('[INFO] Processing building footprints')
        building_footprints = process_buildings(building_footprints, zoning, cbg_sj.crs)
        building_footprints.to_file(processed_footprints_file)
        print('[INFO] Number of buildings: {}'.format(len(building_footprints)))

    # 3) assign buildings to CBGs
    final_footprints_file = os.path.join(fp, 'building_footprints_final')
    if not os.path.exists(final_footprints_file):
        print('[INFO] Loading processed buildings')
        building_footprints = gpd.read_file(processed_footprints_file)
        print('[INFO] Number of buildings: {}'.format(len(building_footprints)))
        
        print('[INFO] Assigning buildings to CBGs') 
        building_footprints = assign_cbgs(building_footprints, cbg_sj)

        # 4) Call gpd.geometry.area on the output from 4) using groupby with the cbg column
        # to get cbg-level area computations for large and small buildings
        print('[INFO] Computing building area')
        building_footprints = compute_area(building_footprints, small_threshold, large_threshold, cbg_sj.crs)
        
        print('[INFO] Generating CBG-level GeoDataFrame')
        cbg_footprints = building_footprints.groupby(['GEOID', 'small', 'large'])[['area']].sum().reset_index()

        # Add back geometry data to CBGs
        cbg_footprints = pd.merge(cbg_footprints, cbg_sj[['GEOID', 'geometry']], how='left', on='GEOID')

        print('[INFO] Exporting building and CBG shapefiles')
        building_footprints.to_file(final_footprints_file)
        cbg_footprints.to_file(os.path.join(fp, 'cbg_inferences'))
    else:
        print('[INFO] Loading building and CBG shapefiles')
        building_footprints = gpd.read_file(final_footprints_file)
        cbg_footprints = gpd.read_file(os.path.join(fp, 'cbg_inferences'))

        # 5) Visualize
        viz(cbg_sj, cbg_footprints, 'small', fp)
        viz(cbg_sj, cbg_footprints, 'large', fp)

    return building_footprints, cbg_footprints

In [10]:
def main_split(cbg_sj, zoning, inferences_dir, size, small_threshold, large_threshold, fp):

    # 1) Loop over each tile to convert the inference np.array to gpd
    # This step outputs a gpd of building footprints: building_footprints
    collected_footprints_file = os.path.join(fp, 'building_inferences')
    if not os.path.exists(collected_footprints_file):
        print('[INFO] Collecting building footprints')
        building_footprints = collect_inferences(inferences_dir, size)
        building_footprints.to_file(collected_footprints_file)
        print('[INFO] Number of buildings: {}'.format(len(building_footprints)))
        
    # 2) Process CBGs individually
    final_footprints_file = os.path.join(fp, 'building_footprints_final')
    if not os.path.exists(final_footprints_file):
        print('[INFO] Loading building footprints')
        building_footprints = gpd.read_file(collected_footprints_file)
        print('[INFO] Number of buildings: {}'.format(len(building_footprints)))
        
        # Assign CBGs
        print('[INFO] Assigning buildings to CBGs') 
        building_footprints = assign_cbgs(building_footprints, cbg_sj)
        building_footprints_processed = gpd.GeoDataFrame(crs=building_footprints.crs)
        
        for cbg in tqdm(cbg_sj.GEOID.unique()):
            cbg_file = os.path.join(fp, 'building_cbg_{}'.format(cbg))
            cbg_geom = cbg_sj.loc[cbg_sj.GEOID == cbg].iloc[0]['geometry']
            
            # Clip building footprints
            building_footprints_cbg = building_footprints.loc[building_footprints['GEOID'] == cbg]
            if len(building_footprints_cbg) > 0:
                if not os.path.exists(cbg_file):
                    # Process cbg footprints
                    zoning_cbg = gpd.clip(zoning, cbg_geom)
                    building_footprints_cbg = process_buildings(building_footprints_cbg, zoning_cbg, cbg_sj.crs, False)

                    # Compute area and filter for buildings that are too small
                    building_footprints_cbg = compute_area(
                        building_footprints_cbg, small_threshold, large_threshold, cbg_sj.crs)

                    # Save 
                    building_footprints_cbg.to_file(cbg_file)

                else:
                    building_footprints_cbg = gpd.read_file(cbg_file)
                    building_footprints_cbg['GEOID'] = cbg
                    
                building_footprints_processed = pd.concat([building_footprints_processed, building_footprints_cbg])
        
        # Save
        print('[INFO] Saving final building footprints')
        building_footprints_processed.to_file(final_footprints_file)
        
        # Combine processed footprints
        print('[INFO] Generating CBG-level GeoDataFrame')
        cbg_footprints = building_footprints_processed.groupby(
            ['GEOID', 'small', 'large'])[['area']].sum().reset_index()

        # Add back geometry data to CBGs
        cbg_footprints = pd.merge(cbg_footprints, cbg_sj[['GEOID', 'geometry']], how='left', on='GEOID')
        
        print('[INFO] Exporting CBG shapefiles')
        cbg_footprints.to_file(os.path.join(fp, 'cbg_inferences'))
        
    print('[INFO] Loading building and CBG shapefiles')
    building_footprints = gpd.read_file(final_footprints_file)
    cbg_footprints = gpd.read_file(os.path.join(fp, 'cbg_inferences'))

    # 5) Visualize
    viz(cbg_sj, cbg_footprints, 'small', fp)
    viz(cbg_sj, cbg_footprints, 'large', fp)

    return building_footprints, cbg_footprints

# 2. Load data

In [11]:
os.getcwd()

'/home/users/avaimar/projects/rgb-footprint-extract/naip_process'

In [12]:
TIF_FP = '/oak/stanford/groups/deho/building_compliance/san_jose_naip_512/raw_tif'
SHAPEFILE_FP = '/oak/stanford/groups/deho/building_compliance/shapefiles'
OAK_FP = '/oak/stanford/groups/deho/building_compliance/'
OUTPUT_FP = os.path.join(OAK_FP, 'outputs', 'cbg-inference')
INFERENCES_DIR = '/oak/stanford/groups/deho/building_compliance/san_jose_naip_512/phase2_superresx2/infer/'

In [13]:
# CBG shapefiles
cbg = gpd.read_file(os.path.join(SHAPEFILE_FP, 'tl_2021_06_bg'))
cbg_scc = cbg.loc[cbg['COUNTYFP'] == '085']

In [14]:
# Get San Jose city SHP
scc_cities = gpd.read_file(os.path.join(SHAPEFILE_FP, 'City_Limits'))
sj_city = scc_cities.loc[scc_cities['NAME'] == 'SAN JOSE']
#sj_city

In [15]:
# Get CBGs in SJ city
cbg_scc = cbg_scc.to_crs(sj_city.crs)
cbg_sj = gpd.clip(cbg_scc, sj_city)

In [16]:
# Verify SJ CBGs 
#plt.clf()
#fig = plt.figure(figsize=(15, 15))
#ax = fig.gca()
#ax.set_aspect('equal')
#cbg_scc.plot(ax=ax, zorder=1, color='white', edgecolor="black")
#cbg_sj.plot(ax=ax, zorder=2, color='red', alpha=0.5)
#sj_city.plot(ax=ax,  edgecolor='blue', zorder=3, alpha=0.3)
#plt.show()

In [17]:
# Zoning data
zoning = gpd.read_file(os.path.join(OAK_FP, 'san_jose_suppl', 'san_jose_Zoning_Districts.geojson'))
zoning = zoning[(zoning['ZONINGABBREV'].str.contains('R-')) | \
                ((zoning['ZONINGABBREV'] == 'A(PD)') & (zoning['PDUSE'] == 'Res'))]
zoning = zoning.to_crs(cbg_sj.crs)

In [18]:
# Generate visualizations
#building_footprints, cbg_footprints = main(cbg_sj=cbg_sj, zoning=zoning, 
#     inferences_dir=INFERENCES_DIR, 
#     size=512*2, small_threshold=115, large_threshold=2000, fp=OUTPUT_FP)

In [ ]:
# Generate visualizations
building_footprints, cbg_footprints = main_split(cbg_sj=cbg_sj, zoning=zoning, 
     inferences_dir=INFERENCES_DIR, 
     size=512*2, small_threshold=180, large_threshold=None, fp=OUTPUT_FP)

[INFO] Loading building footprints
[INFO] Number of buildings: 464725
[INFO] Assigning buildings to CBGs


 50%|█████     | 351/698 [09:57<1:03:43, 11.02s/it]

Visualize raw building footprints

In [ ]:
fp = OUTPUT_FP
collected_footprints_file = os.path.join(fp, 'building_inferences')
building_footprints = gpd.read_file(collected_footprints_file)

In [ ]:
building_footprints = assign_cbgs(building_footprints, cbg_sj)

In [ ]:
cbg = cbg_sj.GEOID.unique()[1]
building_footprints_cbg = building_footprints.loc[building_footprints['GEOID'] == cbg]

In [ ]:
plt.clf()
fig = plt.figure(figsize=(15, 15))
ax = fig.gca()
ax.set_aspect('equal')
cbg_sj.plot(ax=ax, zorder=1, color='white', edgecolor="black")
#cbg_sj.loc[cbg_sj.GEOID == cbg].plot(ax=ax, zorder=2, color='green', edgecolor="black")
building_footprints.plot(ax=ax, zorder=3, alpha=0.5)
plt.show()